In [1]:
import datetime

import requests
from bs4 import BeautifulSoup as bs

---

### 시가총액 3분위, 10분위 기준 구하기

In [2]:
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="
sizeL = []


In [6]:
soup = bs(requests.get(size_market_url_0+"1").text, 'html.parser')
[x.select("td.number")[4].text for x in soup.select("table")[1].select_one("tbody").findAll("tr",{"onmouseout":"mouseOut(this)"}) if isStockByName(x.select_one("a.tltle").text)]


['2,972,952',
 '599,146',
 '395,005',
 '349,060',
 '285,579',
 '252,721',
 '223,496',
 '217,640',
 '207,258',
 '189,376',
 '188,756',
 '173,950',
 '168,355',
 '158,728',
 '155,193',
 '148,108',
 '145,695',
 '139,627',
 '135,411',
 '133,474',
 '127,081',
 '109,746',
 '107,912',
 '102,595',
 '95,702',
 '94,752',
 '93,565',
 '92,994',
 '92,700',
 '81,510',
 '81,245',
 '79,114',
 '72,366',
 '65,637',
 '65,539',
 '59,597',
 '58,431',
 '57,822',
 '51,199',
 '51,025',
 '50,606',
 '50,540',
 '47,562',
 '46,212',
 '45,929',
 '45,830',
 '44,410',
 '40,147',
 '39,966']

In [7]:
for i in range(1, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.select("td.number")[4].text for x in soup.select("table")[1].select_one("tbody").findAll("tr",{"onmouseout":"mouseOut(this)"}) if isStockByName(x.select_one("a.tltle").text)]
    sizeL += [int(x.replace(",","")) for x in tempL]
for i in range(1, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.select("td.number")[4].text for x in soup.select("table")[1].select_one("tbody").findAll("tr",{"onmouseout":"mouseOut(this)"}) if isStockByName(x.select_one("a.tltle").text)]
    sizeL += [int(x.replace(",","")) for x in tempL]


In [8]:
sizeL = sorted(sizeL)

In [9]:
len(sizeL) / 10

266.0

In [11]:
# 3분위까지의 기준
size3 = sizeL[:int(265.6*3)][-1]
print(size3)

# 10분위 까지의 기준
size10 = sizeL[-265:][0]
print(size10)

481
6586


---

### 시가총액 목록 사용하기

In [12]:
# 시가총액 300억 이하 목록 구하기
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="
codeL = []

# 코스피 300억 이하 (후보)
for i in range(1, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

# 코스닥 300억 이하 (후보)
for i in range(1, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
    
# 시가총액 작은 애들부터 보기
codeL.reverse()


In [13]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_하위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = todayDate
# useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_하위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

<b>base code</b>


In [14]:
def getSoupFromCode(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    return soup

def cleanText(text):
    return text.replace("\n", "").replace("\t","").replace("조","").replace(",","")

def getName(soup):
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

def getSoupFromReport(code):
    report_url = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cn=&cmp_cd="
    soup = bs(requests.get(report_url + code).text, 'html.parser')
    return soup

def getJsonFromConsen(code):
    consen_url = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&finGubun=MAIN&frq=0"+\
        "&chartType=svg&cmp_cd="
    return requests.get(consen_url + code).json()

# def increasing(values):
    

---

## 사용할 조건들

- PBR 1 미만
- PER 7 미만
- 시총 300 미만
- PCR 8 미만
- POR 8 미만


---

#### etf 인가

In [15]:
def isETF(soup):
    if(len(soup.select(".e_summary")) == 0):
        return False
    return True

#### etn 인가

In [16]:
def isETN(soup):
    name = getName(soup)
    if("ETN" in name):
        return True
    return False

---

#### 우선주인가


In [17]:
def isFirst(soup):
    name = getName(soup)
    if(name[-1] == "우"):
        return True
    if(name[-2:] in ["우A", "우B", "우C"]):
        return True
    return False

#### 정상적인 주식 (etf, etn, 우선주 제외) 인가

In [18]:
def isStockByName(name):
    if("ETF" in name):
        return False
    if("ETN" in name):
        return False
    if(name[-1] == "우"):
        return False
    if(name[-2:] in ["우A", "우B", "우C"]):
        return False
    return True

#### 시총 300억 이하인가

In [19]:
def marketSumBad(soup):
    marketSum = int(cleanText(soup.select_one("#_market_sum").text))
    if(marketSum <= size3 or marketSum >= size10):
        return False
    return True

#### per 7 미만인가

In [20]:
def perUnder7(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float < 7 and per_float > 0):
            return per_float
        return -1
    except:
        return -1
    

#### pbr 1 미만인가

In [21]:
def pbrUnder1(soup_report):
    try:
        pbr_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"][0]
        pbr_float = float(cleanText(pbr_str))
        if(pbr_float < 1 and pbr_float > 0):
            return pbr_float
        return -1
    except:
        return -1


---

#### 산업평균 per이 양수인가

In [22]:
def per_avg(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "업종PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float > 0):
            return per_float
        return -1
    except:
        return -1

In [23]:
json = getJsonFromConsen("900080")

In [24]:
salesL = [float(cleanText(x["SALES"])) for x in json["JsonData"] if x["SALES"] != ""]

In [16]:
# 영업이익
opL = [float(cleanText(x["OP"])) for x in json["JsonData"] if x["SALES"] != ""]
perL = [float(cleanText(x["PER"])) for x in json["JsonData"] if x["SALES"] != ""]
roeL = [float(cleanText(x["ROE"])) for x in json["JsonData"] if x["SALES"] != ""]

In [17]:
opL

[435.3, 324.4, 294.8, 219.1]

In [18]:
perL

[5.33, 4.76, 5.41, 3.74]

In [19]:
roeL

[8.42, 7.24, 4.96, 4.13]

In [20]:
soup_report.select_one("#cTB511")

NameError: name 'soup_report' is not defined

In [134]:
soup_report.select("#cTB511")

[]

---

## 괜찮은 종목 거르기

In [25]:
good = []
for code in codeL:
    
    # 자원 확보
    soup = getSoupFromCode(code)
    soup_report = getSoupFromReport(code)
    name = getName(soup)
    
    # etf/etn/우선주/300초과 제외
    if(isETF(soup) or isETN(soup) or isFirst(soup) or marketSumBad(soup)):
        continue
        
    # 0<per<7, 0<pbr<1
    per = perUnder7(soup_report)
    pbr = pbrUnder1(soup_report)
    perAvg = per_avg(soup_report)
    if(per == -1 or pbr
       == -1 or perAvg == -1):
        continue
        
    # 출력
    good.append(name + " " + str(per) + " " + str(pbr))
    print(name, per, pbr)

피앤텔 0.47 0.35
차이나그레이트 1.24 0.01
영신금속 3.37 0.46
동일철강 5.34 0.33
PN풍년 3.74 0.5
에스앤씨엔진그룹 2.1 0.05
패션플랫폼 6.85 0.7
신원종합개발 2.33 0.34
원일특강 3.75 0.32
한국컴퓨터 4.97 0.38
바이오로그디바이스 6.68 0.74
코리아에셋투자증권 5.63 0.65
에스에이티 4.86 0.53
케이디켐 6.03 0.56
진양폴리 3.49 0.99
한솔로지스틱스 6.74 0.85
SJM 6.44 0.27
SK가스 4.59 0.43
DGB금융지주 2.43 0.17
다우기술 4.84 0.65
태광산업 6.31 0.2
JB금융지주 2.57 0.25
한국테크놀로지그룹 6.01 0.27
롯데정밀화학 5.46 0.61
영풍 4.14 0.28
코리안리 5.32 0.39
HDC현대산업개발 2.39 0.44
메리츠금융지주 3.63 0.5
대우건설 6.91 0.58
BNK금융지주 2.59 0.18
메리츠화재 5.66 0.69
키움증권 5.55 0.88
GS건설 4.3 0.48
메리츠증권 4.45 0.59
신세계 4.54 0.61
삼성증권 6.26 0.5
한국타이어앤테크놀로지 6.11 0.36
NH투자증권 5.78 0.51
한국금융지주 3.37 0.54
대림산업 4.69 0.53
GS 6.52 0.4
기업은행 3.12 0.22
우리금융지주 2.83 0.26
하나금융지주 3.13 0.26
기아차 6.51 0.41
KB금융 3.91 0.31
신한지주 4.13 0.35


In [22]:
len(good)

11

In [23]:
good

['성우테크론 6.63 0.57',
 '신원종합개발 2.38 0.35',
 '원일특강 3.79 0.32',
 '한국컴퓨터 4.94 0.38',
 '바이오로그디바이스 6.52 0.73',
 '코리아에셋투자증권 5.4 0.62',
 '케이디켐 6.06 0.57',
 '한솔로지스틱스 6.16 0.77',
 '진양산업 5.53 0.92',
 'SJM 5.79 0.24',
 '동원수산 6.55 0.78']

---

## 나의 네이버증권 셋팅하기

In [35]:
!sudo pip3 install selenium -dhtlqwh50

Password:


In [39]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time

ModuleNotFoundError: No module named 'selenium'

In [217]:
# 미리 로그인
driver = webdriver.Chrome('./chromedriver')
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"
driver.get(itemListUrl)

NameError: name 'webdriver' is not defined

In [16]:
for group in range(6, 9):

    # 링크 접속 및 로그인
    itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=%s&type=OVERALL" % str(group)
    print(itemListUrl )
    
    # 기존 관심 종목 삭제
    time.sleep(0.5)
    driver.get(itemListUrl)
    time.sleep(0.5)
    driver.find_element_by_css_selector("#all_select").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
    time.sleep(0.5)
    Alert(driver).accept()
    
    # 적합 종목 리스트 찾기
    if(group == 6):
        useL = score25_code.copy()
    elif(group == 7):
        useL = score23_code.copy()
    elif(group == 8):
        useL = score20_code.copy()
            
    # 관심종목 목록 등록
    for nowCode in useL:
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").clear()
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
        driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)


https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=7&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=8&type=OVERALL
